In [85]:
import sys
import json
import requests
import pandas as pd
import geopandas as gpd

sys.path.insert(1, '../../scripts/modules')
from DizzyAPI import DizzyAPI 


### 1. Set up

In [65]:
properties = pd.read_csv('../../data/raw/raw_domain_properties.csv')
properties = properties[['properties_URL', 'address', 'suburb', 'postcode']]
print(properties.shape)
properties.head()

(9598, 4)


,properties_URL,address,suburb,postcode
0,https://www.domain.com.au/667-glenhuntly-road-...,667 Glenhuntly Road,Caulfield,3162
1,https://www.domain.com.au/upstairs-2c-staley-s...,Upstairs 2C Staley Street,Brunswick,3056
2,https://www.domain.com.au/ground-floor-325-vic...,Ground Floor 325 Victoria Street,Brunswick,3056
3,https://www.domain.com.au/8-chamberlain-road-r...,8 Chamberlain Road,Redcastle,3523
4,https://www.domain.com.au/1-weston-street-balw...,1 Weston Street,Balwyn,3103


In [66]:
ors = DizzyAPI('../../credential/openrouteservice.env', quota=5000, rate=100, num_token=5)

def to_coords(prop):
    headers = {'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8', }
    BASE = 'https://api.openrouteservice.org'
    # BASE = 'http://localhost:8080/ors' use this if local server port 8080
    token = ors.get_token()
    text = f"{prop['address']}, {prop['suburb']}, {prop['postcode']}".replace(' ', '%20')

    # get coding
    if token:
        response = requests.get(f'{BASE}/geocode/search'\
                                              f'?api_key={token}'\  
                                              f'&text={text}'\
                                              '&boundary.country=Aus'\
                                              '&layers=address,venue', 
                                              headers=headers)
    else:
        print(ors.total_request)
        ors.delay(pause=True)
    
    return response

while True:
    print('asd')
    

### 2. Get all properties geocoding


In [67]:
all_coords = []
all_confidence = []
error = []
for i in range(len(properties)):
    response = to_coords(properties.iloc[i])
    try:
        coding = json.loads(response.text)['features'][0]    
        all_coords.append(coding['geometry']['coordinates']) 
        all_confidence.append(coding['properties']['confidence'])
    except Exception:
        all_coords.append([])
        all_confidence.append(-1)
        error.append([properties.iloc[i]['properties_URL'], Exception])

ConnectionError: HTTPSConnectionPool(host='api.openrouteservice.org', port=443): Max retries exceeded with url: /geocode/search?api_key=5b3ce3597851110001cf624823169a9207374801a375d29a5813989a&text=1106/60%20kavanagh%20Street,%20Southbank,%203006&boundary.country=Aus&layers=address,venue (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7fa9a254dea0>: Failed to resolve 'api.openrouteservice.org' ([Errno -3] Temporary failure in name resolution)"))

In [74]:
for i in range(len(properties) - len(all_confidence)):
    all_confidence.append([])
    all_coords.append(-1)


9598

In [75]:
# add to df
properties['coords'] = all_coords
properties['coords_confidence'] = all_confidence

### 3. Save data

In [87]:
# geocoding
properties.to_csv('../../data/raw/properties_geocoding.csv')

# error
pd.DataFrame(error).to_csv('../../data/error_log/geocoding.txt')